Going to play with different Neural Networks to try to model all of the data. I don't seem to have saved a varient of the dataframe where all of the stations are present

In [6]:
import polars as pl
from tqdm import tqdm
import os
import pickle

In [7]:
# # Get list of csv names
# with open("text_files/files.txt", "r") as f:
#     CSVs = f.read()

# CSVs = CSVs.replace("\n    ", " ").replace("%20", "_")[2:-2].split("', '")
# print(CSVs)


In [8]:
# debug. find which files arent in folder that are expected to be
# file_path = f"data/Journey_Data/"
# downloaded_files = set(os.listdir(file_path))
# missing_files = [csv for csv in CSVs if csv not in downloaded_files]
# print(missing_files)

In [9]:
# for testing purposes only select first 50 elements for now (out of 424)
# CSVs = CSVs[:50]

In [12]:
# use CSVs in dir
file_path = f"data/Journey_Data/"
downloaded_files = set(os.listdir(file_path))
CSVs = downloaded_files
print(f"There are {len(CSVs)} files: {CSVs}")

There are 32 files: {'12aJourneyDataExtract15Nov15-27Nov15.csv', '121JourneyDataExtract01Aug2018-07Aug2018.csv', '116JourneyDataExtract27June2018-03July2018.csv', '122JourneyDataExtract08Aug2018-14Aug2018.csv', '109JourneyDataExtract09May2018-15May2018.csv', '11a-Journey-Data-Extract-18Oct15-31Oct15.csv', '129JourneyDataExtract26Sep2018-02Oct2018.csv', '114JourneyDataExtract13June2018-19June2018.csv', '12JourneyDataExtract29Jun2016-05Jul2016.csv', '126JourneyDataExtract05Sep2018-11Sep2018.csv', '10a-Journey-Data-Extract-20Sep15-03Oct15.csv', '10JourneyDataExtract15Jun2016-21Jun2016.csv', '128JourneyDataExtract19Sep2018-25Sep2018.csv', '118JourneyDataExtract11July2018-17July2018.csv', '12bJourneyDataExtract28Nov15-12Dec15.csv', '127JourneyDataExtract12Sep2018-18Sep2018.csv', '112JourneyDataExtract30May2018-05June2018.csv', '117JourneyDataExtract04July2018-10July2018.csv', '124JourneyDataExtract22Aug2018-28Aug2018.csv', '123JourneyDataExtract15Aug2018-21Aug2018.csv', '125JourneyDataExtra

In [ ]:
# Specify your S3 bucket and CSV file paths
bucket_name = 'tfl-cycle-data'
csv_dir = 'Journey_Data/'

# Initialize an empty Polars DataFrame
merged_df = pl.DataFrame()
cols = []

progress_bar = tqdm(CSVs, desc="Processing")

for idx, csv in enumerate(progress_bar):
    file_path = f"temp/{csv}"
    # Download the CSV file from S3
    # s3.download_file(bucket_name, f"{csv_dir}{csv}", file_path)
    file_path = f"data/Journey_Data/{csv}"
    df = pl.read_csv(file_path, skip_rows_after_header=1, ignore_errors=True)
    # print(df.columns)

    # Rename columns and keep only 'Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name'
    renaming_dict = {
        'Number': 'Rental ID',
        'rental ID': 'Rental ID',
        'Rental Id': 'Rental ID',
        'Start date': 'Start Date',
        'StartStation Name': 'Start Station Name',
        'Start station': 'Start Station Name',
        'End date': 'End Date',
        'EndStation Name': 'End Station Name',
        'End station': 'End Station Name'
    }

    for old_col, new_col in renaming_dict.items():
        if old_col in df.columns:
            df = df.rename({old_col: new_col})
    
    cols_to_keep = ['Rental ID', 'Start Date', 'Start Station Name', 'End Date', 'End Station Name']
    df = df.select(cols_to_keep)

    # Convert date columns
    df = df.with_columns([
        pl.col('Start Date').str.strptime(pl.Datetime, strict=False),
        pl.col('End Date').str.strptime(pl.Datetime, strict=False),
        pl.col('Start Station Name').cast(pl.Utf8),
        pl.col('End Station Name').cast(pl.Utf8)
    ])

    # Filter rows (if needed)
    # df = df.filter((pl.col("Start Station Name") == "Waterloo Station 3, Waterloo") |
    #                (pl.col("End Station Name") == "Waterloo Station 3, Waterloo"))
    
    # Concatenate DataFrames
    merged_df = pl.concat([merged_df, df])

    progress_bar.set_description(
        f"Processing Index: {idx+1}/{len(CSVs)}, merged_df is {merged_df.estimated_size()/(1024*1024)}MB large"
    )

Processing Index: 32/32, merged_df is 647.3640289306641MB large: 100%|██████████| 32/32 [00:03<00:00,  8.92it/s] 


In [ ]:
# save to pickle
with open("data/df_temp", 'wb') as f:
    pickle.dump(df, f)

: 

In [32]:
df.head()

Rental ID,Start Date,Start Station Name,End Date,End Station Name
i64,datetime[μs],str,datetime[μs],str
104816169,2021-01-03 12:57:00,"""New Fetter Lane, Holborn""",2021-01-03 13:26:00,"""Salmon Lane, Limehouse"""
104757113,2020-12-30 14:33:00,"""Houndsditch, Aldgate""",2020-12-30 15:00:00,"""Warren Street Station, Euston"""
104749458,2020-12-30 09:08:00,"""Simpson Street, Clapham Juncti…",2020-12-30 09:21:00,"""Ram Street, Wandsworth"""
104788389,2021-01-01 14:59:00,"""Crabtree Lane, Fulham""",2021-01-01 16:29:00,"""Crabtree Lane, Fulham"""
104792584,2021-01-01 20:58:00,"""Crabtree Lane, Fulham""",2021-01-01 21:02:00,"""Crisp Road, Hammersmith"""
